In [ ]:
# %%capture
#!pip install feature_engine

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import set_config

set_config(transform_output="pandas")

df = sns.load_dataset("titanic")
df

* Para realizar cualquier tipo de Preprocesamiento mostraremos 2 procedimientos: `Pandas` y `Scikit-Learn`.

* Ambos permiten el mismo tipo de proceso, la gran diferencia es que Scikit-Learn permite combinar distintos preprocesamientos en un Pipeline y es la manera más `Profesional` de hacer este proceso. 

> Usar Pipelines permitirá evitar errores en procedimientos más complicados que veremos más adelante durante el curso.


## Valores Faltantes

In [ ]:
df.isnull().sum()

In [ ]:
### Pandas
# Veamos cuánto vale el promedio de Edad de los pasajeros del Titanic.
media = df["age"].mean()
mediana = df["age"].median()
print(f"Promedio de Edad: {media}")
print(f'Promedio de Edad con Imputación con Ceros: {df["age"].fillna(0).mean()}')
print(f'Promedio de Edad con Imputación por Media: {df["age"].fillna(media).mean()}')
print(
    f'Promedio de Edad con Imputación por Mediana: {df["age"].fillna(mediana).mean()}'
)

In [ ]:
from sklearn.impute import SimpleImputer

sc = SimpleImputer(strategy="mean")
data_imputed = sc.fit_transform(df[["age"]])  # Ojo con el doble []
data_imputed.isnull().sum()

In [ ]:
mode = df["embark_town"].mode()[0]  # Hablar del tipo de datos que es la Moda.
df["embark_town"].fillna(mode).isnull().sum()

## Outliers

In [ ]:
print(f"Promedio de Tarifas: {df.fare.mean()}")
df["fare"].agg(["min", "max"])

* **lower**: Define la cota inferior.
* **upper**: Define la cota superior.

In [ ]:
clipped_data = df[["fare"]].clip(lower=10, upper=50)
clipped_data.agg(["min", "max"])

In [ ]:
df[["fare"]]

In [ ]:
clipped_data

In [ ]:
from feature_engine.outliers import ArbitraryOutlierCapper, Winsorizer

capper = ArbitraryOutlierCapper(
    max_capping_dict=dict(fare=50), min_capping_dict=dict(fare=10)
)
capper.fit_transform(df[["fare"]])

* **capping_method**: Define la Estragegia a utilizar para el Winsorizer. Ver [Docs](https://feature-engine.trainindata.com/en/latest/api_doc/outliers/Winsorizer.html#feature_engine.outliers.Winsorize).

In [ ]:
win = Winsorizer(capping_method="gaussian")
win.fit_transform(df[["fare"]])

## Variables Categóricas

* **drop_first**: Si es `True` se elimina la primera categoría.

In [ ]:
pd.get_dummies(df["embark_town"], drop_first=False)

In [ ]:
pd.get_dummies(
    df["embark_town"], drop_first=True
)  # No considera los Nulos como otra categoría...

* **sort**: Usar `True` ya que coloca las categorías en orden. Además de esta manera se comporta igual que `OrdinalEncoder` de `Scikit-Learn`.

In [ ]:
pd.DataFrame(pd.factorize(df["embark_town"], sort=True)[0], columns=["new_column"])

* **sparse_output**: Se debe fijar como False para poder ver el output como Pandas
* **drop**: Se debe colocar `"first"` o el nombre de *una sóla categoría* a eliminar.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

ohe = OneHotEncoder(drop="first", sparse_output=False)
ohe.fit_transform(df[["embark_town"]])

In [ ]:
ohe = OneHotEncoder(
    drop=["Queenstown"], sparse_output=False
)  # También se puede colocar np.nan.
ohe.fit_transform(df[["embark_town"]])

In [ ]:
oe = OrdinalEncoder()
oe.fit_transform(df[["embark_town"]])

## Escalamiento

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

sc = StandardScaler()
data = sc.fit_transform(df[["fare"]])
data.agg(["mean", "std"])

In [ ]:
mms = MinMaxScaler()
mms.fit_transform(df[["fare"]]).agg(["min", "max"])